In [1]:
%matplotlib inline
%load_ext memory_profiler

import os, sys, glob, re, math, pickle
import phate,scprep,magic,meld
import graphtools as gt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time,random,datetime
from sklearn import metrics
from sklearn import model_selection
from scipy import sparse
from scipy.stats import mannwhitneyu, tiecorrect, rankdata
from statsmodels.stats.multitest import multipletests
import scanpy as sc
import scvelo as scv
from adjustText import adjust_text
import warnings
from matplotlib import gridspec



# settings
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=9)
plt.rcParams['savefig.dpi']=600
sc.settings.verbosity=2
sc._settings.ScanpyConfig.n_jobs=-1
sns.set_style("ticks")

In [2]:
# fps
dfp = '/home/cl2292/project/SCA1_snRNAseq/Human/data/'
pfp = '/home/cl2292/project/SCA1_snRNAseq/Human/results_20230402/'
pdfp = '/home/cl2292/project/SCA1_snRNAseq/Human/data/processed/'
sc.settings.figdir = pfp

In [3]:
# settings
plt.rc('font', size = 8)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=8)
plt.rcParams['savefig.dpi']=600
sc.settings.verbosity=2
sc._settings.ScanpyConfig.n_jobs=-1



In [4]:
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_Ctrl_imp.h5ad' # for full, can maybe get away with ~300G
    %memit wt = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    
# Load Data

if True :
    start = time.time()
    backed=None # None if not
    fname='230502_SCA1_imp.h5ad' # for full, can maybe get away with ~300G
    %memit mut = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))

peak memory: 43226.09 MiB, increment: 43026.75 MiB
loaded @230519.11:44:47
took 39.23-s to load data
peak memory: 73473.84 MiB, increment: 30247.66 MiB
loaded @230519.11:45:19
took 32.09-s to load data


In [7]:
genes = ['GABRA6','EOMES','ATP2A3','GAD2','LGI2','ALDH1L1','GDF10','PDGFRA',
         'MOG','CX3CR1','FLT1','LUM', 'CA8', 'PPP1R17','NTN1',
         'NXPH1', 'PTPRK', 'NRG1','PAX2']

In [8]:
wt = wt[:,genes]
mut = mut[:,genes]

In [9]:
adata = wt.concatenate(mut, batch_key='concat')

In [ ]:
# # save data objects
# adata.write(os.path.join(pdfp,'230519_ctrl+SCA1_imp.h5ad'))
# print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

In [ ]:
sc.pl.umap(adata, color =genes, size = 5, cmap = 'magma', save = '230519_umap_genes_before_magic')

In [ ]:
sc.pl.umap(adata, color =genes, size = 5, cmap = 'magma',  layer = 'imputed', save = '230519_umap_genes_after_magic')